In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect(':memory:')

In [3]:
cursor = conn.cursor()

In [7]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS mytable (
    col1 VARCHAR(255),
    col2 TEXT,
    col3 INT,
    col4 FLOAT
)
""")

In [9]:
cursor.execute("PRAGMA table_info('mytable')")

In [10]:
cursor.execute("PRAGMA table_info('mytable')").fetchall()

[(0, 'col1', 'VARCHAR(255)', 0, None, 0),
 (1, 'col2', 'TEXT', 0, None, 0),
 (2, 'col3', 'INT', 0, None, 0),
 (3, 'col4', 'FLOAT', 0, None, 0)]

In [11]:
cursor.execute("PRAGMA table_info('mytable')").fetchone()

(0, 'col1', 'VARCHAR(255)', 0, None, 0)

In [18]:
cursor.execute('DROP TABLE mytable')

In [19]:
cursor.execute("PRAGMA table_info('mytable')").fetchone()

In [20]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS mytable (
    col1 TEXT NOT NULL,
    col2 TEXT UNIQUE,
    col3 INT NOT NULL UNIQUE,
    col4 FLOAT UNIQUE NOT NULL
)
""")

In [21]:
# row number, column name, type, not nullable, default, primary key
cursor.execute("PRAGMA table_info('mytable')").fetchall()

[(0, 'col1', 'TEXT', 1, None, 0),
 (1, 'col2', 'TEXT', 0, None, 0),
 (2, 'col3', 'INT', 1, None, 0),
 (3, 'col4', 'FLOAT', 1, None, 0)]

In [24]:
conn.executescript("""
    CREATE TABLE IF NOT EXISTS mytable (
        col1 TEXT NOT NULL,
        col2 TEXT NOT NULL UNIQUE,
        col3 INTEGER NOT NULL UNIQUE,
        col4 REAL UNIQUE
    );
    CREATE TABLE IF NOT EXISTS mytable2 (
        col1 TEXT NOT NULL,
        col2 TEXT NOT NULL UNIQUE,
        col3 INTEGER NOT NULL UNIQUE,
        col4 REAL UNIQUE
    );
""")

In [28]:
cursor.execute("PRAGMA table_info('mytable')").fetchall()

[(0, 'col1', 'TEXT', 1, None, 0),
 (1, 'col2', 'TEXT', 0, None, 0),
 (2, 'col3', 'INT', 1, None, 0),
 (3, 'col4', 'FLOAT', 1, None, 0)]

In [31]:
cursor.execute("""
    INSERT INTO mytable VALUES ('foo', 'bar', 10, 1.2)
""")

IntegrityError: UNIQUE constraint failed: mytable.col4

In [30]:
cursor.execute("""
    SELECT * FROM mytable
""").fetchall()

[('foo', 'bar', 10, 1.2)]

In [34]:
cursor.execute("""
    INSERT INTO mytable VALUES ('foo', 'barbaz', '50', '2.5')
""")

In [35]:
cursor.execute("""
    SELECT * FROM mytable
""").fetchall()

[('foo', 'bar', 10, 1.2), ('foo', 'barbaz', 50, 2.5)]

In [36]:
col1 = 'eenie'
col2 = 'meenie'
col3 = 'one'
col4 = 9.2

In [38]:
cursor.execute(f"""
    INSERT INTO mytable VALUES ('{col1}', '{col2}', '{col3}', '{col4}')
""")

In [39]:
cursor.execute("""
    SELECT * FROM mytable
""").fetchall()

[('foo', 'bar', 10, 1.2),
 ('foo', 'barbaz', 50, 2.5),
 ('eenie', 'meenie', 1, 9.2)]

In [40]:
cursor.execute(f"""
    INSERT INTO mytable(col2, col3, col4) 
    VALUES ('{col2}', '{col3}', '{col4}')
""")

IntegrityError: NOT NULL constraint failed: mytable.col1

SQL injection

In [ ]:
col1 = "'); DROP TABLE mytable; "

qmarks

In [43]:
cursor.execute(f"""
    INSERT INTO mytable VALUES (?, ?, ?, ?)
""", ('a', 'b', 4, 9.3))

In [44]:
cursor.execute("""
    SELECT * FROM mytable
""").fetchall()

[('foo', 'bar', 10, 1.2),
 ('foo', 'barbaz', 50, 2.5),
 ('eenie', 'meenie', 1, 9.2),
 ('a', 'b', 4, 9.3)]

In [45]:
cursor.executemany("""
    INSERT INTO mytable VALUES (?, ?, ?, ?)
""", [('c', 'd', 5, 10), ('e', 'f', 6, 11.2)]).fetchall()

[]

In [46]:
cursor.execute("""
    SELECT * FROM mytable
""").fetchall()

[('foo', 'bar', 10, 1.2),
 ('foo', 'barbaz', 50, 2.5),
 ('eenie', 'meenie', 1, 9.2),
 ('a', 'b', 4, 9.3),
 ('c', 'd', 5, 10.0),
 ('e', 'f', 6, 11.2)]

In [48]:
cursor.execute("""
    SELECT *
    FROM mytable
""").fetchall()

[('foo', 'bar', 10, 1.2),
 ('foo', 'barbaz', 50, 2.5),
 ('eenie', 'meenie', 1, 9.2),
 ('a', 'b', 4, 9.3),
 ('c', 'd', 5, 10.0),
 ('e', 'f', 6, 11.2)]

In [50]:
cursor.execute("""
    SELECT col1, col4
    FROM mytable
""").fetchall()

[('foo', 1.2),
 ('foo', 2.5),
 ('eenie', 9.2),
 ('a', 9.3),
 ('c', 10.0),
 ('e', 11.2)]

In [3]:
import pandas as pd

In [4]:
from sqlalchemy import create_engine

In [5]:
engine = create_engine('sqlite:///') # in-memory sqlite database

In [6]:
with engine.connect() as conn:
    pd.read_sql('SELECT * FROM mytable', conn)

OperationalError: (sqlite3.OperationalError) no such table: mytable
[SQL: SELECT * FROM mytable]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [7]:
# sqlite://
# / (relative path)
# / (absolute path)
engine2 = create_engine('sqlite:////home/nbgrader/source/sql-exercises/'
                        'exercises.db')

In [6]:
with engine2.connect() as conn:
    df = pd.read_sql('SELECT * FROM aisles', conn)
    print(df)

     aisle_id                       aisle
0           1       prepared soups salads
1           2           specialty cheeses
2           3         energy granola bars
3           4               instant foods
4           5  marinades meat preparation
..        ...                         ...
129       130    hot cereal pancake mixes
130       131                   dry pasta
131       132                      beauty
132       133  muscles joints pain relief
133       134  specialty wines champagnes

[134 rows x 2 columns]


In [7]:
with engine2.connect() as conn:
    df = pd.read_sql('SELECT aisle FROM aisles', conn)
df

,aisle
0,prepared soups salads
1,specialty cheeses
2,energy granola bars
3,instant foods
4,marinades meat preparation
...,...
129,hot cereal pancake mixes
130,dry pasta
131,beauty
132,muscles joints pain relief


In [8]:
with engine2.connect() as conn:
    df = pd.read_sql("""SELECT COUNT(*) FROM aisles""", conn)
df

,COUNT(*)
0,134


In [9]:
with engine2.connect() as conn:
    df = pd.read_sql("""SELECT COUNT(aisle) FROM aisles""", conn)
df

,COUNT(aisle)
0,134


In [10]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT COUNT(aisle) 
    FROM aisles
    LIMIT 10
    """, conn)
df

,COUNT(aisle)
0,134


In [11]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    LIMIT 10
    """, conn)
df

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


In [12]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    WHERE aisle_id > 3
    LIMIT 5
    """, conn)
df

,aisle_id,aisle
0,4,instant foods
1,5,marinades meat preparation
2,6,other
3,7,packaged meat
4,8,bakery desserts


In [13]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    WHERE aisle_id > 3 AND aisle = "other"
    LIMIT 5
    """, conn)
df

,aisle_id,aisle
0,6,other


In [14]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT aisle
    FROM aisles
    WHERE aisle_id = 7
    """, conn)
df

,aisle
0,packaged meat


In [15]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    WHERE aisle = "other"
    """, conn)
df

,aisle_id,aisle
0,6,other


In [16]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT aisle_id
    FROM aisles
    WHERE aisle = "other"
    """, conn)
df

,aisle_id
0,6


In [17]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT aisle_id
    FROM aisles
    WHERE aisle LIKE "packaged %"
    LIMIT 10
    """, conn)
df

,aisle_id
0,7
1,15
2,21
3,32
4,49
5,123


In [18]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT COUNT(*)
    FROM aisles
    WHERE aisle LIKE "packaged %"
    """, conn)
df

,COUNT(*)
0,6


In [19]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    WHERE aisle LIKE "packaged %"
    """, conn)
df

,aisle_id,aisle
0,7,packaged meat
1,15,packaged seafood
2,21,packaged cheese
3,32,packaged produce
4,49,packaged poultry
5,123,packaged vegetables fruits


In [20]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    WHERE aisle LIKE "Packaged %"
    """, conn)
df

,aisle_id,aisle
0,7,packaged meat
1,15,packaged seafood
2,21,packaged cheese
3,32,packaged produce
4,49,packaged poultry
5,123,packaged vegetables fruits


In [21]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    WHERE aisle LIKE "%meat%"
    """, conn)
df

,aisle_id,aisle
0,5,marinades meat preparation
1,7,packaged meat
2,14,tofu meat alternatives
3,34,frozen meat seafood
4,95,canned meat seafood
5,96,lunch meat
6,122,meat counter


In [22]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    WHERE aisle NOT LIKE "%meat%"
    """, conn)
df

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,6,other
...,...,...
122,130,hot cereal pancake mixes
123,131,dry pasta
124,132,beauty
125,133,muscles joints pain relief


In [23]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT UPPER(aisle)
    FROM aisles
    LIMIT 10
    """, conn)
df

,UPPER(aisle)
0,PREPARED SOUPS SALADS
1,SPECIALTY CHEESES
2,ENERGY GRANOLA BARS
3,INSTANT FOODS
4,MARINADES MEAT PREPARATION
5,OTHER
6,PACKAGED MEAT
7,BAKERY DESSERTS
8,PASTA SAUCE
9,KITCHEN SUPPLIES


In [24]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    ORDER BY aisle
    LIMIT 10
    """, conn)
df

,aisle_id,aisle
0,101,air fresheners candles
1,66,asian foods
2,82,baby accessories
3,102,baby bath body care
4,92,baby food formula
5,8,bakery desserts
6,17,baking ingredients
7,97,baking supplies decor
8,132,beauty
9,27,beers coolers


In [25]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    ORDER BY aisle ASC
    LIMIT 10
    """, conn)
df

,aisle_id,aisle
0,101,air fresheners candles
1,66,asian foods
2,82,baby accessories
3,102,baby bath body care
4,92,baby food formula
5,8,bakery desserts
6,17,baking ingredients
7,97,baking supplies decor
8,132,beauty
9,27,beers coolers


In [26]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    ORDER BY aisle DESC
    LIMIT 10
    """, conn)
df

,aisle_id,aisle
0,120,yogurt
1,62,white wines
2,115,water seltzer sparkling water
3,47,vitamins supplements
4,60,trash bags liners
5,125,trail mix snack mix
6,128,tortillas flat bread
7,14,tofu meat alternatives
8,94,tea
9,88,spreads


In [27]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    ORDER BY aisle DESC, aisle_id
    LIMIT 10
    """, conn)
df

,aisle_id,aisle
0,120,yogurt
1,62,white wines
2,115,water seltzer sparkling water
3,47,vitamins supplements
4,60,trash bags liners
5,125,trail mix snack mix
6,128,tortillas flat bread
7,14,tofu meat alternatives
8,94,tea
9,88,spreads


In [28]:
engine2.table_names()

['aisles', 'departments', 'players', 'products', 'reactions', 'transactions']

In [29]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM transactions
    LIMIT 10
    """, conn)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,01/12/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,01/12/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,01/12/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,01/12/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,01/12/2010 8:34,1.69,13047.0,United Kingdom


In [30]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM transactions
    GROUP BY Country
    LIMIT 10
    """, conn)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536389,22941,CHRISTMAS LIGHTS 10 REINDEER,6,01/12/2010 10:03,8.50,12431.0,Australia
1,C538971,22153,ANGEL DECORATION STARS ON DRESS,-48,15/12/2010 11:39,0.42,12865.0,Austria
2,539500,72802B,OCEAN SCENT CANDLE IN JEWELLED BOX,54,20/12/2010 11:02,3.81,NaN,Bahrain
3,537026,84375,SET OF 20 KIDS COOKIE CUTTERS,12,03/12/2010 16:35,2.10,12395.0,Belgium
4,550201,22423,REGENCY CAKESTAND 3 TIER,16,15/04/2011 10:25,10.95,12769.0,Brazil
5,546533,20886,BOX OF 9 PEBBLE CANDLES,12,14/03/2011 13:53,1.95,15388.0,Canada
6,538002,22690,DOORMAT HOME SWEET HOME BLUE,2,09/12/2010 11:48,7.95,14932.0,Channel Islands
7,538826,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,14/12/2010 12:58,2.55,12370.0,Cyprus
8,545072,22930,BAKING MOULD HEART MILK CHOCOLATE,18,28/02/2011 8:43,2.55,12781.0,Czech Republic
9,538003,22847,BREAD BIN DINER STYLE IVORY,8,09/12/2010 12:05,14.95,12429.0,Denmark


In [31]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT Country, COUNT(*) AS product_count
    FROM transactions
    GROUP BY Country
    LIMIT 10
    """, conn)
df

,Country,product_count
0,Australia,1259
1,Austria,401
2,Bahrain,19
3,Belgium,2069
4,Brazil,32
5,Canada,151
6,Channel Islands,758
7,Cyprus,622
8,Czech Republic,30
9,Denmark,389


In [8]:
# country, average quantity and number of unique invoices per country of the
# 10 countries with the most invoices and that have more than 10 unique
# invoices sorted by decreasing number of unique invoices
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT Country, COUNT(DISTINCT(InvoiceNo)) AS invoice_count
    FROM transactions
    GROUP BY Country
    LIMIT 10
    """, conn)
df

,Country,invoice_count
0,Australia,69
1,Austria,19
2,Bahrain,4
3,Belgium,119
4,Brazil,1
5,Canada,6
6,Channel Islands,33
7,Cyprus,20
8,Czech Republic,5
9,Denmark,21


In [9]:
# country and number of unique invoices per country of the 10 countries with
# the most invoices and that have more than 10 unique invoices sorted by
# decreasing number of unique invoices
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT Country, COUNT(DISTINCT(InvoiceNo)) AS invoice_count
    FROM transactions
    GROUP BY Country
    HAVING invoice_count > 10
    ORDER BY invoice_count DESC
    LIMIT 10
    """, conn)
df

,Country,invoice_count
0,United Kingdom,23494
1,Germany,603
2,France,461
3,EIRE,360
4,Belgium,119
5,Spain,105
6,Netherlands,101
7,Switzerland,74
8,Portugal,71
9,Australia,69


In [10]:
# country, average quantity and number of unique invoices per country of the
# 10 countries with the most invoices and that have more than 10 unique
# invoices sorted by decreasing number of unique invoices
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT 
        Country, 
        AVG(Quantity), 
        COUNT(DISTINCT(InvoiceNo)) AS invoice_count
    FROM transactions
    GROUP BY Country
    HAVING invoice_count > 10
    ORDER BY invoice_count DESC
    LIMIT 10
    """, conn)
df

,Country,AVG(Quantity),invoice_count
0,United Kingdom,8.605486,23494
1,Germany,12.369458,603
2,France,12.911067,461
3,EIRE,17.403245,360
4,Belgium,11.189947,119
5,Spain,10.589814,105
6,Netherlands,84.406580,101
7,Switzerland,15.147353,74
8,Portugal,10.651745,71
9,Australia,66.444003,69


In [38]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM products
    LIMIT 10
    """, conn)
df

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


In [39]:
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM transactions
    JOIN products
    ON transactions.StockCode = products.product_id
    LIMIT 10
    """, conn)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,product_id,product_name,aisle_id,department_id
0,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,01/12/2010 8:26,7.65,17850.0,United Kingdom,22752,Organic Pink Lady (Cripps) Apple Bag,100,21
1,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,01/12/2010 8:26,4.25,17850.0,United Kingdom,21730,Refreshing Mint Vital Energy Black Tea Blend B...,94,7
2,536366,22633,HAND WARMER UNION JACK,6,01/12/2010 8:28,1.85,17850.0,United Kingdom,22633,1200mg Fish Oil,47,11
3,536366,22632,HAND WARMER RED POLKA DOT,6,01/12/2010 8:28,1.85,17850.0,United Kingdom,22632,Quick-Dissolving Sinus Homeopathic Medicine Ta...,47,11
4,536367,22745,POPPY'S PLAYHOUSE BEDROOM,6,01/12/2010 8:34,2.10,13047.0,United Kingdom,22745,Curry Recipe Seitan,14,20
5,536367,22748,POPPY'S PLAYHOUSE KITCHEN,6,01/12/2010 8:34,2.10,13047.0,United Kingdom,22748,Black Rice,63,9
6,536367,22749,FELTCRAFT PRINCESS CHARLOTTE DOLL,8,01/12/2010 8:34,3.75,13047.0,United Kingdom,22749,Toaster Pops Apple,52,1
7,536367,22310,IVORY KNITTED MUG COSY,6,01/12/2010 8:34,1.65,13047.0,United Kingdom,22310,Natural Energy Drink,64,7
8,536367,22623,BOX OF VINTAGE JIGSAW BLOCKS,3,01/12/2010 8:34,4.95,13047.0,United Kingdom,22623,Sprouted Spicy Fiesta Seed Mix,125,19
9,536367,22622,BOX OF VINTAGE ALPHABET BLOCKS,2,01/12/2010 8:34,9.95,13047.0,United Kingdom,22622,Organic Sunflower Kernels,117,19


In [41]:
# stock code, product_name, unit price of the first 10 items
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT t.StockCode, p.product_name, t.UnitPrice
    FROM transactions AS t
    JOIN products AS p
    ON t.StockCode = p.product_id
    LIMIT 10
    """, conn)
df

,StockCode,product_name,UnitPrice
0,22752,Organic Pink Lady (Cripps) Apple Bag,7.65
1,21730,Refreshing Mint Vital Energy Black Tea Blend B...,4.25
2,22633,1200mg Fish Oil,1.85
3,22632,Quick-Dissolving Sinus Homeopathic Medicine Ta...,1.85
4,22745,Curry Recipe Seitan,2.10
5,22748,Black Rice,2.10
6,22749,Toaster Pops Apple,3.75
7,22310,Natural Energy Drink,1.65
8,22623,Sprouted Spicy Fiesta Seed Mix,4.95
9,22622,Organic Sunflower Kernels,9.95


In [43]:
# stock code, product name and unitprice of the first item of invoices with
# total price (sum(quantity*unitprice)) greater than 1000, sorted by 
# decreasing unit price, limited to the 10 items with the largest unit price
with engine2.connect() as conn:
    df = pd.read_sql("""
    SELECT t.StockCode, p.product_name, t.UnitPrice
    FROM transactions AS t
    JOIN products AS p
    ON t.StockCode = p.product_id
    GROUP BY t.InvoiceNo
    HAVING SUM(t.Quantity*t.UnitPrice) > 1000
    ORDER BY UnitPrice DESC
    LIMIT 10
    """, conn)
df

,StockCode,product_name,UnitPrice
0,22502,Razor Chrome (Includes 2 Cartridges),649.50
1,22827,Organic Black Mission Figs,145.00
2,22503,Asian Sesame Kit,29.95
3,22761,Peanut Butter Squared Fun Size,24.95
4,21217,Cranberry Trail Mix,20.79
5,22830,Light Grape Juice Beverage Concord Grapes,19.94
6,22830,Light Grape Juice Beverage Concord Grapes,19.84
7,22688,"Pro-Health Multi-Protection Mouthwash, Alcohol...",17.88
8,22215,Bleach Clean Linen Concentrated,16.98
9,22846,Ionic Zinc 50 mg Drops,16.95
